In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("student_performance_data.csv")
def load_dataset(file_path):
    """
    Load a dataset and display basic information about it.
    
    Parameters:
    file_path (str): Path to the dataset file.
    
    Returns:
    DataFrame: The loaded dataset.
    """
    try:
        # Try to infer the file type from extension
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(file_path)
        else:
            print(f"Unsupported file type: {file_path}")
            return None
        
        print("\nDataset Shape:", df.shape)
        print("\nFirst 5 Rows:")
        print(df.head())
        print("\nData Types:")
        print(df.dtypes)
        print("\nSummary Statistics:")
        print(df.describe())
        print("\nMissing Values per Column:")
        print(df.isnull().sum())
        return df
    
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None
    

In [2]:
dataset = load_dataset('student_performance_data.csv')


Dataset Shape: (1000, 14)

First 5 Rows:
   Student ID  Gender  Age Parent Education Level  Hours Studied Per Week  \
0        1001  Female   15            High School                     4.6   
1        1002    Male   18           Some College                    14.5   
2        1003    Male   16             Bachelor's                     8.9   
3        1004  Female   18           Some College                    11.3   
4        1005  Female   16               Graduate                    13.6   

   Sleep Hours Per Night  Extracurricular Activities  Math Score  \
0                    6.9                         6.0          53   
1                    7.8                         4.6         100   
2                    7.6                         7.4          94   
3                    6.9                         6.4         100   
4                    5.9                         5.4          79   

   Science Score  English Score  Overall GPA  Absence Rate Technology Access  \
0     